In [ ]:
'''
extractive sumarization using unsupervised learning 
i use cosine similarity because of chosen approach (sentences as vectors)
cosine similarity measures the angle between two vectors 
if the angle is 0 the sentences are similar 

Input article → split into sentences → remove stop words → build a similarity matrix → 
generate rank based on matrix → pick top N sentences for summary.

'''

In [51]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jolajakobowska/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [97]:
def prepare_article(file_name):
    file = open(file_name, "r")
    file_data = file.readlines() # full text of the article - 
    article = file_data[0].split(". ")
    sentences = []
    
    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
        
    sentences.pop()
    print("Given text: \n", file_data, "\n")
    return sentences 
    

In [98]:
#creates vectors from sentences and returns cosine distances extracted from 1 to later get the similarity 
def vectors_from_sentences(sentence_1, sentence_2, stopwords=None):
    if stopwords is None:
        stopwords = []
        
    sentence_1 = [w.lower() for w in sentence_1] #w - word
    sentence_2 = [w.lower() for w in sentence_2]
    
    everything = list(set(sentence_1 + sentence_2))
    
    vector_1 = [0] *len(everything)
    vector_2 = [0] *len(everything)
    
    #first sentence 
    for w in sentence_1:
        if w in stopwords:
            continue 
        vector_1[everything.index(w)] += 1 
  
    #second sentence
    for w in sentence_2:
        if w in stopwords:
            continue 
        vector_2[everything.index(w)] += 1
  
    return 1 - cosine_distance(vector_1, vector_2)

In [99]:
#create a matrix based on cosine distance
def do_matrix_of_similarity(sentences, stop_words):
    similarity_matrix = np.zeros((len(sentences), len(sentences))) #get an array of zeros (empty) to compare two sentences
    
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue 
            similarity_matrix[idx1][idx2] = vectors_from_sentences(sentences[idx1],sentences[idx2])
            
    return similarity_matrix

In [100]:
def give_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    
    #prepare the data
    sentences = prepare_article(file_name) 
    
    #make the matrix of vectors and return cosine distance
    sentence_simmilarity_matrix = do_matrix_of_similarity(sentences, stop_words) 
    
    sentence_simmilarity_graph = nx.from_numpy_array(sentence_simmilarity_matrix)
    scores = nx.pagerank(sentence_simmilarity_graph)
    
    ranked_sentence = sorted(((scores[i], s) for i,s in enumerate(sentences)), reverse=True)
    
    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
        
    ready_summary = ". ".join(summarize_text)
    print("Summarry: \n", ready_summary)


In [101]:
give_summary( "msft.txt", 1)

Given text: 
 ['In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses. The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, "With AI being the defining technology of our time, it is transforming live